Three datasets on an adult literacy program are available with this document. 

The dataset Tutor.xlsx stores tutor information (tutor ID, certificate date, and tutor status). Before the tutors teach any student, they are required to complete a certification class offered by the agency. Their certificate completion dates are stored under the CertDate column. The column TutorStatus reports each tutor’s current status as Active (currently serving), Temp Stop (temporarily not working), or Dropped (dropped out of the program).

The dataset Student.xlsx stores student information (student ID, student group, and read score). Students are required to complete an assessment interview before they attend any tutoring session. The assessment results are recorded under the ReadScore column and are shared with the tutors. Each student belongs to a student group (StudentGroup). 
When matched with a student, a tutor meets with the student on one-on-one basis.  

The dataset Match_History.xlsx stores the match data (match ID, tutor ID, student ID, start of tutoring, and end of tutoring). Students change tutors if their learning style does not match the tutor’s tutoring style. Tutors may have more than one student. The StartDate column shows the beginning of tutoring for an individual student. The EndDate column shows the end of tutoring. An empty cell of the EndDate column represents ongoing tutoring.

Write a python script to perform the following queries :
1.	Which tutors have a Dropped status and have achieved their certification after 4/01/2018?
2.	What is the average length of time a student stayed (or has stayed) in the program? You may use the current date for ongoing tutoring. 
3.	Identify all students who have been matched in 2018 with a tutor whose status is Temp Stop.
4.	List the Read scores of students who were ever taught by tutors whose status is Dropped.
5.	List the tutors who taught two or more students. 
6.	Display a list of all students, their read score, their tutors, and tutors status. Store this information in a file names Student_Tutor.xlsx. 
7.	For each student group, list the number of tutors who have been matched with that group.
8.	List all active students who started in May and June. 
9.	Find students who have not been tutored yet.
10.	Find tutors who did not tutor any students. 

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
tutor = pd.read_excel("Tutor.xlsx")
student = pd.read_excel("Student.xlsx")
match_history = pd.read_excel("Match_History.xlsx")

## Examining data frames visually.

In [3]:
tutor.head()

,TutorID,CertDate,TutorStatus
0,100,2018-01-05,Active
1,101,2018-01-05,Temp Stop
2,102,2018-01-05,Dropped
3,103,2018-05-22,Active
4,104,2018-05-22,Active


In [4]:
student.head()

,StudentID,StudentGroup,ReadScore
0,3000,3,2.3
1,3001,2,5.6
2,3002,3,1.3
3,3003,1,3.3
4,3004,2,2.7


In [5]:
match_history

,MatchID,TutorID,StudentID,StartDate,EndDate
0,1,100,3000,2018-01-10,NaT
1,2,101,3001,2018-01-15,2018-05-15
2,3,102,3002,2018-02-10,2018-03-01
3,4,106,3003,2018-05-28,NaT
4,5,103,3004,2018-06-01,2018-06-15
5,6,104,3005,2018-06-01,2018-06-28
6,7,104,3006,2018-06-01,NaT
7,8,100,3001,2018-05-28,NaT


## Question 1: Which tutors have a Dropped status and have achieved their certification after 04/01/2018.

In [6]:
tutor[(tutor["TutorStatus"] == "Dropped") & (tutor["CertDate"] > "2018-04-01")]

,TutorID,CertDate,TutorStatus
7,107,2018-05-01,Dropped


## Question 2: What is the average length of time a student stayed (or has stayed) in the program? You may use the current date for ongoing tutoring.

In [7]:
today = dt.datetime.today().strftime('%Y-%m-%d')
match_history.loc[match_history["EndDate"].isna(), "EndDate"] = today
match_history["EndDate"] = match_history["EndDate"].astype("datetime64")
print((match_history["EndDate"] - match_history["StartDate"]).mean())

680 days 18:00:00


## Question 3: Identify all students who have been matched in 2018 with a tutor whose status is Temp Stop.

In [8]:
merged_m_t = pd.merge(match_history, tutor, how = "left", on = "TutorID")
merged_m_t[(merged_m_t["TutorStatus"] == "Temp Stop") & (merged_m_t["StartDate"].dt.year == 2018)]

,MatchID,TutorID,StudentID,StartDate,EndDate,CertDate,TutorStatus
1,2,101,3001,2018-01-15,2018-05-15,2018-01-05,Temp Stop


## Question 4: List the Read scores of students who were ever taught by tutors whose status is Dropped.

In [9]:
merged_m_t_s = pd.merge(merged_m_t, student, how = "left", on = "StudentID")
merged_m_t_s[merged_m_t_s["TutorStatus"] == "Dropped"][["StudentID", "TutorStatus", "ReadScore"]]

,StudentID,TutorStatus,ReadScore
2,3002,Dropped,1.3


## Question 5: List the tutors who taught two or more students.

In [10]:
tutor_count = match_history["TutorID"].value_counts()
print(tutor_count[tutor_count >= 2].index.values)

[100 104]


## Question 6: Display a list of all students, their read score, their tutors, and tutors status. Store this information in a file names Student_Tutor.xlsx.

In [11]:
merged_s_m = pd.merge(student, match_history, how = "left", on = "StudentID")
merged_s_m_t = pd.merge(merged_s_m, tutor, how = "left", on = "TutorID")
all_student = merged_s_m_t[["StudentID", "ReadScore", "TutorID", "TutorStatus"]]
all_student.to_excel("Student_Tutor.xlsx")
all_student

,StudentID,ReadScore,TutorID,TutorStatus
0,3000,2.3,100.0,Active
1,3001,5.6,101.0,Temp Stop
2,3001,5.6,100.0,Active
3,3002,1.3,102.0,Dropped
4,3003,3.3,106.0,Active
5,3004,2.7,103.0,Active
6,3005,4.8,104.0,Active
7,3006,7.8,104.0,Active
8,3007,1.5,NaN,NaN


## Question 7: For each student group, list the number of tutors who have been matched with that group.

In [12]:
merged_s_m_t_2 = merged_s_m_t.dropna()
merged_s_m_t_2.groupby("StudentGroup")["TutorID"].count()

StudentGroup
1    1
2    3
3    3
4    1
Name: TutorID, dtype: int64

## Question 8: List all active students who started in May and June. 

In [13]:
active_student = merged_s_m_t[merged_s_m_t["EndDate"] == today]
active_student_started_M_J = active_student[(active_student["StartDate"].dt.month == 5) | 
                                            (active_student["StartDate"].dt.month == 6)]
active_student_started_M_J[["StudentID", "StartDate", "EndDate"]]

,StudentID,StartDate,EndDate
2,3001,2018-05-28,2021-12-01
4,3003,2018-05-28,2021-12-01
7,3006,2018-06-01,2021-12-01


## Question 9: Find students who have not been tutored yet.

In [14]:
untutor_student = merged_s_m_t[merged_s_m_t["TutorID"].isna()]
untutor_student[["StudentID", "TutorID", "TutorStatus"]]

,StudentID,TutorID,TutorStatus
8,3007,NaN,NaN


## Question 10: Find tutors who did not tutor any students. 

In [15]:
tutor[~tutor["TutorID"].isin(merged_m_t_s["TutorID"])]

,TutorID,CertDate,TutorStatus
5,105,2018-05-22,Temp Stop
7,107,2018-05-01,Dropped
